# Lock-in: Control y adquisición de señales

In [1]:
#Generación de gráficas y .csv 
#Datos de referencia: fase, frecuencia y amplitud
#Datos de ganacia y constante de tiempo: sensibilidad y constante de tiempo
#Datos de display y output: Elección de canal (CH1), Display: R / CH2: Theta

In [2]:
import pyvisa
import time
import numpy as np
import csv
import time

In [3]:
def adquirir(q):
    lockin.write(q)
    data=''
    try:
        while True:
            b=lockin.read_bytes(1)
            data=data+str(b)
    except:
       # print('Lectura terminada')
        pass
    data=data.replace('b','')
    data=data.replace('\'','')
    data=data.replace(r'\r','')
    data=float(data)
    return data

In [4]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())
lockin = rm.open_resource('ASRL1::INSTR', timeout=100)

('ASRL1::INSTR',)


Datos de entrada

In [7]:
n = 5
tiempo_estabilizacion=300
n_barridos = 1
lista_freq = [1, 10, 100]
# Ajustar Fase de Referencia
lockin.write("PHAS {}".format(0))
time.sleep(0.2)
# Ajustar Amplitud de Referencia
lockin.write("SLVL {}".format(4.24))
time.sleep(1)
# Ajustar Constante de tiempo a 1 s
lockin.write("OELT {}".format(10))
time.sleep(1)
# Ajustar Sensibilidad a 50 mV/nA
#lockin.write("SENS {}".format(22))

Realizar n barridos de captura de datos

In [8]:
for barrido in range(1, n_barridos+1):
    start_time = time.time()
    print('barrido ' + str(barrido))
    # Para cada frecuencia, tomar n muestras
    # para cada frecuencia en la lista de frecuencias
    tabla_muestras=[]
    for freq in lista_freq:
        print('freq ' + str(freq))
        #Ajustar frecuencia de referencia
        lockin.write("FREQ {}".format(freq))
        time.sleep(tiempo_estabilizacion)
        lista_muestras=[]
        lista_muestras.append(freq)
        # repetir n veces
        for i in range(n):
            # adquirir muestra y asignarlo en R y theta
            R = adquirir ('OUTP? 3')
            # print('R: ' + str(R))
            time.sleep(1)
            lista_muestras.append(R)
            theta = adquirir ('OUTP? 4')
            # print('Theta: ' + str(theta))
            time.sleep(1)
            lista_muestras.append(theta)
        tabla_muestras.append(lista_muestras) 
        duracion_barrido= time.time() - start_time
    with open("Prueba-1-1" + str(barrido) + "-" +str(duracion_barrido) + ".csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(tabla_muestras)

barrido 1
freq 1
freq 10
freq 100


In [45]:
import os

In [46]:
os.getcwd()

'C:\\Users\\khemi\\Lock-in Control\\SR830_LockInAmplifier'